## Example: Which Hacker News comments generated the most discussion?
___

Now we're ready to work through an example on a real dataset. Today, we're going to be using the Hacker News dataset, which contains information on stories & comments from the Hacker News social networking site. I want to know which comments on the site generated the most replies.

First, just like yesterday, we need to get our environment set up. I already know that I want the "comments" table, so I'm going to look at the first couple of rows of that to get started.

In [1]:
# import package with helper functions 
import bq_helper

# create a helper object for this dataset
hacker_news = bq_helper.BigQueryHelper(active_project="bigquery-public-data",
                                   dataset_name="hacker_news")

# print the first couple rows of the "comments" table
hacker_news.head("comments")

,id,by,author,time,time_ts,text,parent,deleted,dead,ranking
0,2701393,5l,5l,1309184881,2011-06-27 14:28:01+00:00,And the glazier who fixed all the broken windo...,2701243,None,None,0
1,5811403,99,99,1370234048,2013-06-03 04:34:08+00:00,Does canada have the equivalent of H1B/Green c...,5804452,None,None,0
2,21623,AF,AF,1178992400,2007-05-12 17:53:20+00:00,"Speaking of Rails, there are other options in ...",21611,None,None,0
3,10159727,EA,EA,1441206574,2015-09-02 15:09:34+00:00,Humans and large livestock (and maybe even pet...,10159396,None,None,0
4,2988424,Iv,Iv,1315853580,2011-09-12 18:53:00+00:00,I must say I reacted in the same way when I re...,2988179,None,None,0


By looking at the documentation, I learned that the "parent" column has information on the comment that each comment was a reply to and the "id" column has the unique id used to identify each comment. So I can group by the "parent" column and count the "id" column in order to figure out the number of comments that were made as responses to a specific comment. 

Because I'm more interested in popular comments than unpopular comments, I'm also only going to return the groups that have more than ten id's in them. In other words, I'm only going to look at comments that had more than ten comment replies to them.

In [2]:
# query to pass to 
query = """SELECT parent, COUNT(id) as counts
            FROM `bigquery-public-data.hacker_news.comments`
            GROUP BY parent
            HAVING COUNT(id) > 10
        """

Now that our query is ready, let's run it (safely!) and store the results in a dataframe: 

In [3]:
# the query_to_pandas_safe method will cancel the query if
# it would use too much of your quota, with the limit set 
# to 1 GB by default
popular_stories = hacker_news.query_to_pandas_safe(query)

And, just like yesterday, we have a dataframe we can treat like any other data frame:

In [4]:
popular_stories.head()

,parent,f0_
0,6427895,46
1,202918,44
2,8120079,148
3,9016949,38
4,7075537,51


Looks good! From here I could do whatever further analysis or visualization I'd like. 

> **Why is the column with the COUNT() data called f0_**? It's called this because COUNT() is the first (and in our case, only) aggregate function we used in this query. If we'd used a second one, it would be called "f1\_", the third would be called "f2\_", and so on. We'll learn how to name the output of aggregate functions later this week.

And that should be all you need to get started writing your own kernels with GROUP BY... WHERE and COUNT!

# Scavenger hunt
___

Now it's your turn! Here's the questions I would like you to get the data to answer:

* How many stories (use the "id" column) are there of each type (in the "type" column) in the full table?
* How many comments have been deleted? (If a comment was deleted the "deleted" column in the comments table will have the value "True".)
* **Optional extra credit**: read about [aggregate functions other than COUNT()](https://cloud.google.com/bigquery/docs/reference/standard-sql/functions-and-operators#aggregate-functions) and modify one of the queries you wrote above to use a different aggregate function.

In order to answer these questions, you can fork this notebook by hitting the blue "Fork Notebook" at the very top of this page (you may have to scroll up). "Forking" something is making a copy of it that you can edit on your own without changing the original.

In [5]:
# Your code goes here 

query = """ select type, count(id) as counts from `bigquery-public-data.hacker_news.full` group by type """
stories = hacker_news.query_to_pandas_safe(query)
print(stories)


      type    counts
0  comment  19564602
1    story   3717642
2      job     12929
3  pollopt     12164
4     poll      1788


In [6]:

query = """ select type,count(deleted) as deleted_count from `bigquery-public-data.hacker_news.full` where deleted = True and type = 'comment' group by type """

deleted = hacker_news.query_to_pandas_safe(query)
deleted
print('Total {} comments were deleted.'.format(deleted['deleted_count'][0]))


Total 521586 comments were deleted.


In [7]:
# EXTRA CREDIT

query = """ select type, avg(score) as avg_score from `bigquery-public-data.hacker_news.full` group by type having sum(score) > 0 """
score = hacker_news.query_to_pandas_safe(query)
score

,type,avg_score
0,story,12.017612
1,job,1.896543
2,pollopt,47.000843
3,poll,28.489097


Please feel free to ask any questions you have in this notebook or in the [Q&A forums](https://www.kaggle.com/questions-and-answers)! 

Also, if you want to share or get comments on your kernel, remember you need to make it public first! You can change the visibility of your kernel under the "Settings" tab, on the right half of your screen.